In [11]:
## Random Forest Classifier on our dataset

import pandas as pd
import numpy as np

df = pd.read_json('datasets/final_user_joke_dataset.json')

df.head()



,joke_id,USF1000_rating,USF1001_rating,USF1002_rating,datasource_joke_id,jokeText,jester,reddit,stupidstuff,toxic,...,sexist,age_24,age_26,gender_Female,gender_Male,ethnicity_Middle Eastern,ethnicity_South East Asian,country_India,country_Iran,location_San Francisco
0,RE5tz52q,1,8,0,5tz52q,I hate how you cant even say black paint anymo...,0,1,0,0.948757,...,0.009227,0.033333,0.266667,0.033333,0.266667,0.000000,0.300000,0.300000,0.000000,0.300000
1,RE5tz4dd,1,8,0,5tz4dd,What's the difference between a Jew in Nazi Ge...,0,1,0,0.740485,...,0.010509,0.033333,0.266667,0.033333,0.266667,0.000000,0.300000,0.300000,0.000000,0.300000
2,RE5tz319,0,9,8,5tz319,I recently went to America....\n...and being t...,0,1,0,0.713800,...,0.008724,0.000000,0.566667,0.266667,0.300000,0.266667,0.300000,0.300000,0.266667,0.566667
3,RE5tz2wj,3,9,0,5tz2wj,"Brian raises his hand and says, “He’s in Heave...",0,1,0,0.863763,...,0.010060,0.100000,0.300000,0.100000,0.300000,0.000000,0.400000,0.400000,0.000000,0.400000
4,RE5tz1pc,5,3,8,5tz1pc,You hear about the University book store worke...,0,1,0,0.556188,...,0.006886,0.166667,0.366667,0.433333,0.100000,0.266667,0.266667,0.266667,0.266667,0.533333


In [12]:
curr_user_traits = {
    "age": 26,
    "gender": "Male",
    "ethnicity": "South East Asian",
    "country": "Iran",
    "location" : "San Francisco"
}


col_names_to_use = [ str(key) + '_' + str(value) for key, value in curr_user_traits.items()]

# col_names_to_use = set(col_names_to_use) - set(df.columns)

# col_names_to_use = list(col_names_to_use)

col_names_to_use


['age_26',
 'gender_Male',
 'ethnicity_South East Asian',
 'country_Iran',
 'location_San Francisco']

In [13]:
df.columns

Index(['joke_id', 'USF1000_rating', 'USF1001_rating', 'USF1002_rating',
       'datasource_joke_id', 'jokeText', 'jester', 'reddit', 'stupidstuff',
       'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate',
       'anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise',
       'misogynist', 'sexist', 'age_24', 'age_26', 'gender_Female',
       'gender_Male', 'ethnicity_Middle Eastern', 'ethnicity_South East Asian',
       'country_India', 'country_Iran', 'location_San Francisco'],
      dtype='object')

In [14]:
for col in col_names_to_use:
    result_df = df[df[col] > 0.5]

# result_df['USF1002_rating'] = result_df['USF1002_rating'].apply(lambda x: 1 if x > 6 else 0)

In [15]:
result_df['USF1002_rating']

2       8
4       8
7       9
21     10
24      5
       ..
181     8
182     0
183     5
184     0
186     4
Name: USF1002_rating, Length: 93, dtype: int64

In [16]:
columns_to_keep = col_names_to_use

columns_to_keep.extend([
    # 'USF1000_rating', 
    # 'USF1001_rating', 
    # 'USF1002_rating',
    # "jester",
    # "reddit",
    # "stupidstuff",
    # 'joke_id',
    # 'jokeText', 
    'jester', 
    'reddit', 
    'stupidstuff',
    'toxic', 
    'severe_toxic', 
    'obscene', 
    'threat', 
    'insult', 
    'identity_hate',
    'anger', 
    'disgust', 
    'fear', 
    'joy', 
    'neutral', 
    'sadness', 
    'surprise',
    'misogynist', 
    'sexist', 
])

# label_df = temp_df['USF1002_rating'].apply(lambda x: 1 if x > 0.5 else 0)

# result_df = result_df[result_df['USF1002_rating']]

label_df = result_df['USF1002_rating']

result_df = result_df[columns_to_keep]

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(result_df, label_df , test_size=0.2, random_state=42)

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier


# params = {'max_depth': [3, 4, 5], # Go smaller
#           'criterion': ['entropy', 'gini'],
#           'n_estimators': [115, 130, 150], # Go smaller, 3/5/10,x,
#           'max_features': [None]
#         }

# params = {'max_depth': [None, 5, 10, 20, 30], 
#           'criterion': ['absolute_error', 'squared_error'],
#           'n_estimators': [115, 130, 150, 200], 
#           'max_features': [None, 'log2']
#         }

params = {'max_depth': [5, 10, 9],
          'criterion': ['squared_error'],
          'n_estimators': [50, 150, 200]
        }


folds = 5
rf = RandomForestRegressor()

clf = GridSearchCV(rf, params, cv=folds, n_jobs=-1)

clf.fit(X_train, y_train)
print(clf.best_params_)

{'criterion': 'squared_error', 'max_depth': 5, 'n_estimators': 50}


In [19]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

# Train the model with the best parameters obtained

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

rf = RandomForestRegressor(**clf.best_params_, random_state=42, n_jobs=-1, oob_score=True)

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

accuracy = rf.score(X_test, y_test)
print("Accuracy->", accuracy)

Accuracy-> 0.9595838622915546


In [14]:
# # implent K-Nearest Neighbors classifier (KNN) algorithm to predict the rating of a joke for a user

# from sklearn.neighbors import KNeighborsClassifier

# knn = KNeighborsClassifier(n_neighbors=5)

# knn.fit(X_train, y_train)

# y_pred = knn.predict(X_test)

# accuracy = knn.score(X_test, y_test)

# print("Accuracy->", accuracy)

In [15]:
# # implement gradient boosting classifier (GBC) algorithm to predict the rating of a joke for a user

# from sklearn.ensemble import GradientBoostingClassifier

# gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)

# gbc.fit(X_train, y_train)

In [20]:
# implent K-Nearest Neighbors regressor (KNN) algorithm to predict the rating of a joke for a user

from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=5)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print("MSE->", mse)

accuracy = knn.score(X_test, y_test)

print("Accuracy->", accuracy)

MSE-> 10.193684210526316
Accuracy-> 0.010242065626680952


In [21]:
# Implement Gradient Boosting Regressor

from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor()

gbr.fit(X_train, y_train)

y_pred = gbr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print("MSE->", mse)

accuracy = gbr.score(X_test, y_test)

print("Accuracy->", accuracy)

MSE-> 0.4387490274709753
Accuracy-> 0.9573995699523878
